In [156]:
import math
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta, date
import pandas as pd
from fredapi import Fred
import openpyxl
from openpyxl.styles import Font, Border, Side

path = '/Users/christynatalisa/Library/CloudStorage/ShareFile-ShareFile/Shared Folders/1011 - 2024 Trade Desk CN/Practice/20 - Trade Group/800 - SY Desk/SPX/Matrix Q/SPXW Option Prices.xlsx'
SPXW = yf.Ticker("^SPX")

# Define the date range
start_date = datetime(2024, 11, 2)
end_date = datetime(2024, 11, 26)

# Loop over each date in the range
current_date = start_date
while current_date <= end_date:
    # Get the next row in the sheet
    workbook = openpyxl.load_workbook(path)
    sheetName = 'Pricing'
    sheet = workbook[sheetName]
    
    next_row = sheet.max_row + 1
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        if all(cell.value is None for cell in row):
            next_row = row[0].row
            break

    s = SPXW.history(period='1d')['Close'][0]

    # SPXW volatility
    data = yf.download('^SPX', period='1y')
    data['Daily Return'] = data['Close'].pct_change()
    vol = data['Daily Return'].std() * np.sqrt(252)

    # Risk-free interest rate
    fredAPIKey = 'a4dbd27388238842e83d39b9959f2759'
    fred = Fred(api_key=fredAPIKey)
    riskFreeRateSeries = fred.get_series('DGS1')
    riskFreeRate = riskFreeRateSeries[-1]/100
    r = riskFreeRate

    # Option Jun 25 Put 5700
    k = 5700
    t = ((datetime(2025,6,20) - current_date).days)/365.25
    optJun25P5700 = SPXW.option_chain('2025-06-20')
    putJun25P5700 = optJun25P5700.puts
    x = putJun25P5700[putJun25P5700['strike'] == 5700]
    lst = x.values.tolist()
    realPricePut5700 = lst[0][3]

    # Option Jan 26 Put 5300
    k2 = 5300
    t2 = ((datetime(2026,1,16) - current_date).days)/365.25
    optJan16P5300 = SPXW.option_chain('2026-01-16')
    putJan16P5300 = optJan16P5300.puts
    x2 = putJan16P5300[putJan16P5300['strike'] == 5300]
    lst2 = x2.values.tolist()
    realPricePut5300 = lst2[0][3]

    def normCdf(x):
        return(1.0 + math.erf(x/math.sqrt(2.0)))/2.0

    def putCalculation(s,r,k,t,vol):
        d1 = (math.log(s/k)+(r + 0.5 * vol **2)*t)/(vol * math.sqrt(t))
        d2 = d1 - (vol * math.sqrt(t))
        Put = k * math.exp(-r*t) * normCdf(-d2) - s * normCdf(-d1)
        return(Put)

    pricePut5700 = putCalculation(s,r,k,t,vol)
    pricePut5300 = putCalculation(s,r,k2,t2,vol)

    # Create the cells in the sheet for the current date
    title_cell = sheet.cell(row=next_row, column=1)
    title_cell.value = current_date.date()  # Use current_date.date() to store only the date
    title_cell.font = Font(bold=True)

    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))

    title_cell.border = thin_border

    putJun255700Cell = sheet.cell(row=next_row, column=2)
    putJan265300Cell = sheet.cell(row=next_row, column=5)
    putJun255700Cell.value = round(pricePut5700,2)
    putJan265300Cell.value = round(pricePut5300,2)
    putJun255700Cell.border = thin_border
    putJan265300Cell.border = thin_border

    putJun255700CellRealPrice = sheet.cell(row=next_row, column=3)
    putJan265300CellRealPrice = sheet.cell(row=next_row, column=6)
    putJun255700CellRealPrice.value = realPricePut5700
    putJan265300CellRealPrice.value = realPricePut5300
    putJun255700CellRealPrice.border = thin_border
    putJan265300CellRealPrice.border = thin_border

    deltaPutJun255700 = sheet.cell(row=next_row, column=4)
    deltaPutJan265300 = sheet.cell(row=next_row, column=7)
    deltaPutJun255700.value = round((pricePut5700-realPricePut5700),2)
    deltaPutJan265300.value = round((pricePut5300-realPricePut5300),2)
    deltaPutJun255700.border = thin_border
    deltaPutJan265300.border = thin_border

    # Save the workbook after processing this date
    workbook.save(path)

    # Move to the next date
    current_date += timedelta(days=1)

# Print the results after all dates are processed
print("The Black Schole price of the Put Option Jun 25 123 for SPXW is: $", round(pricePut5700,4),'and the real price is $',realPricePut5700)
print("\nThe Black Schole price of the Put Option Jan 26 110 for SPXW is: $", round(pricePut5300,4),'and the real price is $',realPricePut5300)


/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/149425013.py:31: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

[*********************100%***********************]  1 of 1 completed
/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/149425013.py:42: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/149425013.py:31: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use 

The Black Schole price of the Put Option Jun 25 123 for SPXW is: $ 61.2616 and the real price is $ 120.8

The Black Schole price of the Put Option Jan 26 110 for SPXW is: $ 30.7848 and the real price is $ 162.2


/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/149425013.py:42: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [158]:
import yfinance as yf
import pandas as pd

# Define the date range
start_date = "2024-11-02"
end_date = "2024-11-26"

# Define the options tickers
options_tickers = ["SPX260116P05300000", "SPX250620P05700000"]  # Replace with correct tickers if needed.


option_data1 = yf.download("SPX260116P05300000", start=start_date, end=end_date, interval='1d')
option_data2 = yf.download("SPX250620P05700000", start=start_date, end=end_date, interval='1d')

option_data_df1 = pd.DataFrame(option_data1)
option_data_df1.fillna(method='ffill', inplace=True)  # Forward fill
# option_data_df1.fillna(method='bfill', inplace=True)  # Backward fill

option_data_df2 = pd.DataFrame(option_data2)
option_data_df2.fillna(method='ffill', inplace=True)  # Forward fill
# option_data_df1
option_data_df2


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/1875528705.py:16: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/1875528705.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPX250620P05700000,SPX250620P05700000,SPX250620P05700000,SPX250620P05700000,SPX250620P05700000,SPX250620P05700000
Date,,,,,,
2024-11-04 00:00:00+00:00,223.800003,223.800003,230.979996,221.000000,230.979996,1079
2024-11-05 00:00:00+00:00,198.279999,198.279999,213.630005,198.279999,213.630005,293
2024-11-06 00:00:00+00:00,151.509995,151.509995,161.729996,151.509995,155.600006,327
2024-11-07 00:00:00+00:00,136.039993,136.039993,144.690002,135.979996,144.690002,161
2024-11-08 00:00:00+00:00,130.869995,130.869995,139.589996,130.869995,139.589996,144
2024-11-11 00:00:00+00:00,130.690002,130.690002,131.949997,127.160004,127.160004,252
2024-11-12 00:00:00+00:00,133.100006,133.100006,139.339996,130.889999,133.619995,495
2024-11-13 00:00:00+00:00,130.130005,130.130005,138.339996,127.300003,138.339996,0


In [159]:
import math
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
from fredapi import Fred

# Initialize the SPXW ticker and Fred API
SPXW = yf.Ticker("^SPX")
fredAPIKey = 'a4dbd27388238842e83d39b9959f2759'
fred = Fred(api_key=fredAPIKey)

# Define the date range
start_date = datetime(2024, 11, 2)
end_date = datetime(2024, 11, 26)

# Create an empty DataFrame to store the results
columns = [
    'Date', 'PricePut5700', 'RealPricePut5700', 'DeltaPut5700',
    'PricePut5300', 'RealPricePut5300', 'DeltaPut5300'
]
results = pd.DataFrame(columns=columns)

# Loop over each date in the range
current_date = start_date
while current_date <= end_date:
    s = SPXW.history(period='1d')['Close'][0]

    # Calculate SPXW volatility
    data = yf.download('^SPX', period='1y')
    data['Daily Return'] = data['Close'].pct_change()
    vol = data['Daily Return'].std() * np.sqrt(252)

    # Risk-free interest rate
    riskFreeRateSeries = fred.get_series('DGS1')
    riskFreeRate = riskFreeRateSeries[-1] / 100
    r = riskFreeRate

    # Option Jun 25 Put 5700
    k = 5700
    t = ((datetime(2025, 6, 20) - current_date).days) / 365.25
    optJun25P5700 = SPXW.option_chain('2025-06-20')
    putJun25P5700 = optJun25P5700.puts
    x = putJun25P5700[putJun25P5700['strike'] == 5700]
    lst = x.values.tolist()
    realPricePut5700 = lst[0][3]

    # Option Jan 26 Put 5300
    k2 = 5300
    t2 = ((datetime(2026, 1, 16) - current_date).days) / 365.25
    optJan16P5300 = SPXW.option_chain('2026-01-16')
    putJan16P5300 = optJan16P5300.puts
    x2 = putJan16P5300[putJan16P5300['strike'] == 5300]
    lst2 = x2.values.tolist()
    realPricePut5300 = lst2[0][3]

    def normCdf(x):
        return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

    def putCalculation(s, r, k, t, vol):
        d1 = (math.log(s / k) + (r + 0.5 * vol ** 2) * t) / (vol * math.sqrt(t))
        d2 = d1 - (vol * math.sqrt(t))
        Put = k * math.exp(-r * t) * normCdf(-d2) - s * normCdf(-d1)
        return Put

    pricePut5700 = putCalculation(s, r, k, t, vol)
    pricePut5300 = putCalculation(s, r, k2, t2, vol)

    # Calculate deltas
    deltaPut5700 = pricePut5700 - realPricePut5700
    deltaPut5300 = pricePut5300 - realPricePut5300

    # Create a DataFrame for the new row
    new_row = pd.DataFrame([{
        'Date': current_date.date(),
        'PricePut5700': round(pricePut5700, 2),
        'RealPricePut5700': realPricePut5700,
        'DeltaPut5700': round(deltaPut5700, 2),
        'PricePut5300': round(pricePut5300, 2),
        'RealPricePut5300': realPricePut5300,
        'DeltaPut5300': round(deltaPut5300, 2)
    }])

    # Append the new row to the results DataFrame
    results = pd.concat([results, new_row], ignore_index=True)

    # Move to the next date
    current_date += timedelta(days=1)

# Print the final DataFrame
# print(results)

# Optionally save the DataFrame to an Excel file
# results.to_excel('SPXW_Option_Prices.xlsx', index=False)


/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/1847183584.py:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

[*********************100%***********************]  1 of 1 completed
/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/1847183584.py:36: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/38/nx0s9dz55h1dpkx5p0wgg46m0000gn/T/ipykernel_14103/1847183584.py:85: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To re

In [160]:
option_data_df1 = option_data_df1.rename(columns={'Close': 'RealPricePut5300'})
option_data_df2 = option_data_df2.rename(columns={'Close': 'RealPricePut5700'})


In [ ]:
# Reset the index
option_data_df1 = option_data_df1.reset_index()

# Ensure 'Date' column is in datetime format
option_data_df1['Date'] = pd.to_datetime(option_data_df1['Date'])

# Set 'Date' as the new index
option_data_df1 = option_data_df1.set_index('Date')

option_data_df2 = option_data_df2.reset_index()

# Ensure 'Date' column is in datetime format
option_data_df2['Date'] = pd.to_datetime(option_data_df2['Date'])

# Set 'Date' as the new index
option_data_df2 = option_data_df2.set_index('Date')

# Format the Date index to 'YYYY-MM-DD'
option_data_df2.index = option_data_df2.index.strftime('%Y-%m-%d')


In [167]:
results.columns

Index(['Date', 'PricePut5700', 'RealPricePut5700', 'DeltaPut5700',
       'PricePut5300', 'RealPricePut5300', 'DeltaPut5300'],
      dtype='object')

In [ ]:
option_data_df2.columns = option_data_df2.columns.droplevel('Ticker')
option_data_df2.columns = option_data_df2.columns.rename(None)

option_data_df1.columns = option_data_df1.columns.droplevel('Ticker')
option_data_df1.columns = option_data_df1.columns.rename(None)

In [168]:
results['Date'] = pd.to_datetime(results['Date'])

# Set the 'date' column as the index
results = results.set_index('Date')
# Change the index format to 'yyyy-mm-dd'
results.index = results.index.strftime('%Y-%m-%d')
results.update(option_data_df1, overwrite=True)
results.update(option_data_df2, overwrite=True)

In [165]:
option_data_df1

,Adj Close,RealPricePut5300,High,Low,Open,Volume
Date,,,,,,
2024-11-04 00:00:00+00:00,208.210007,208.210007,208.699997,208.210007,208.699997,52
2024-11-06 00:00:00+00:00,158.399994,158.399994,160.710007,158.399994,160.710007,27
2024-11-07 00:00:00+00:00,150.750000,150.750000,150.750000,150.750000,150.750000,1
2024-11-08 00:00:00+00:00,145.199997,145.199997,145.199997,145.199997,145.199997,1
2024-11-15 00:00:00+00:00,162.199997,162.199997,162.199997,161.399994,161.399994,3


In [169]:
import plotly.express as px
import pandas as pd

# Assuming 'results' is your DataFrame containing the data
# Make sure the index is set as 'Date' and it is in datetime format
results.index = pd.to_datetime(results.index)

# Create a Plotly figure excluding the Delta columns
fig = px.line(results, 
              x=results.index,  # Use the DataFrame index as the x-axis
              y=['PricePut5700', 'RealPricePut5700', 
                 'PricePut5300', 'RealPricePut5300'],
              labels={'value': 'Option Price', 'index': 'Date'},
              title="Option Prices Over Time")

# Add a date range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),  # Enable range slider
        type='date'  # Make sure the x-axis is a date type
    ),
    xaxis_rangeslider_visible=True,  # Make the range slider visible
    title="SPXW Option Prices",
    xaxis_title="Date",
    yaxis_title="Price",
    legend_title="Options"
)

# Show the figure
fig.show()


In [155]:
import plotly.express as px
import pandas as pd

# Assuming 'results' is your DataFrame containing the data
# Make sure the index is set as 'Date' and it is in datetime format
results.index = pd.to_datetime(results.index)

# Create a Plotly figure excluding the Delta columns
fig = px.line(results, 
              x=results.index,  # Use the DataFrame index as the x-axis
              y=['PricePut5700', 'RealPricePut5700', 
                 'PricePut5300', 'RealPricePut5300'],
              labels={'value': 'Option Price', 'index': 'Date'},
              title="Option Prices Over Time")

# Update the layout to customize the range slider and hide unnecessary elements
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=False,  # Enable range slider
            bgcolor='grey',  # Set the range slider's background color to grey
            borderwidth=0,  # Remove border around the slider
            thickness=0.05,  # Set the thickness of the range slider
        ),
        type='date'  # Make sure the x-axis is a date type
    ),
    xaxis_rangeslider_visible=True,  # Make the range slider visible
    title="SPXW Option Prices",
    xaxis_title="Date",
    yaxis_title="Price",
    plot_bgcolor='rgba(0,0,0,0)',  # Make the plot background transparent
)

# Show the figure
fig.show()
